In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

Generate synthetic data for sample

In [13]:
num_samples = 2000  
start_time = datetime(2024, 2, 1, 0, 0)  


timestamps = []
load_weight = []
motor_temp = []
vibration = []
operating_hours = []
error_code = []


for i in range(num_samples):
    timestamps.append(start_time + timedelta(minutes=i*5)) 
    load_weight.append(round(random.uniform(500, 5000), 2))  
    motor_temp.append(round(random.uniform(30, 90), 2))  
    vibration.append(round(random.uniform(0.2, 5.0), 2))  
    operating_hours.append(round(random.uniform(100, 5000), 1))  
    error_code.append(random.choice([0, 1]) if vibration[-1] > 4.5 or motor_temp[-1] > 85 else 0)  


df = pd.DataFrame({
    'Timestamp': timestamps,
    'Load_Weight (kg)': load_weight,
    'Motor_Temperature (°C)': motor_temp,
    'Vibration_Level (mm/s)': vibration,
    'Operating_Hours': operating_hours,
    'Error_Code': error_code  
})


df.to_csv("synthetic_crane_data.csv", index=False)

print(df.head())

            Timestamp  Load_Weight (kg)  Motor_Temperature (°C)  \
0 2024-02-01 00:00:00           1523.52                   87.31   
1 2024-02-01 00:05:00           4183.36                   60.60   
2 2024-02-01 00:10:00           1563.33                   51.77   
3 2024-02-01 00:15:00           4392.60                   40.21   
4 2024-02-01 00:20:00           4427.35                   30.27   

   Vibration_Level (mm/s)  Operating_Hours  Error_Code  
0                    3.64           4695.5           0  
1                    1.85           3855.5           0  
2                    2.84           3247.4           0  
3                    3.54           4506.4           0  
4                    3.38           4085.4           0  


Analyis

In [3]:
df['Vibration_MA'] = df['Vibration_Level (mm/s)'].rolling(window=10, min_periods=1).mean()
df['Temp_MA'] = df['Motor_Temperature (°C)'].rolling(window=10, min_periods=1).mean()
#trend analysis

In [4]:
df['Vibration_Change'] = df['Vibration_Level (mm/s)'].diff()
df['Temp_Change'] = df['Motor_Temperature (°C)'].diff()
#anamoly detection

In [5]:
df['Cumulative_Hours'] = df['Operating_Hours'].cumsum()
#failure probability

In [6]:
df['Vibration_Std'] = df['Vibration_Level (mm/s)'].rolling(window=10, min_periods=1).std()
df['Temp_Std'] = df['Motor_Temperature (°C)'].rolling(window=10, min_periods=1).std()
#Volatility Detection

In [7]:
df['Failure_Probability'] = (
    (df['Vibration_Level (mm/s)'] / df['Vibration_Level (mm/s)'].max()) * 0.4 +
    (df['Motor_Temperature (°C)'] / df['Motor_Temperature (°C)'].max()) * 0.4 +
    (df['Operating_Hours'] / df['Operating_Hours'].max()) * 0.2
)
#Combining Features

In [8]:
df.to_csv("engineered_crane_data.csv", index=False)
print(df.head())


            Timestamp  Load_Weight (kg)  Motor_Temperature (°C)  \
0 2024-02-01 00:00:00           3019.76                   56.08   
1 2024-02-01 00:05:00           3942.66                   58.12   
2 2024-02-01 00:10:00           3528.53                   49.38   
3 2024-02-01 00:15:00           4702.80                   77.61   
4 2024-02-01 00:20:00           3945.19                   31.15   

   Vibration_Level (mm/s)  Operating_Hours  Error_Code  Vibration_MA  \
0                    0.71           2063.6           0      0.710000   
1                    1.66           4358.6           0      1.185000   
2                    1.36           3385.5           0      1.243333   
3                    0.37            628.1           0      1.025000   
4                    3.53           3763.1           0      1.526000   

     Temp_MA  Vibration_Change  Temp_Change  Cumulative_Hours  Vibration_Std  \
0  56.080000               NaN          NaN            2063.6            NaN   
1  5

Training the data

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv("engineered_crane_data.csv")
df = df.drop(columns=['Timestamp'])

Define Features and Target

In [11]:

X = df.drop(columns=['Error_Code'])  
y = df['Error_Code']  

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Train XGBoost Model

In [12]:
#
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)


y_pred_xgb = xgb_model.predict(X_test)


print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1817
           1       0.51      0.45      0.48       183

    accuracy                           0.91      2000
   macro avg       0.73      0.70      0.72      2000
weighted avg       0.91      0.91      0.91      2000



In [15]:
import joblib

# ... (your model training code) ...
# Initialize and train XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on test set
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate XGBoost
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))
# Save the trained model
joblib.dump(xgb_model, "best_xgboost.pkl")
joblib.dump(scaler, "scaler.pkl")

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1817
           1       0.51      0.45      0.48       183

    accuracy                           0.91      2000
   macro avg       0.73      0.70      0.72      2000
weighted avg       0.91      0.91      0.91      2000



['scaler.pkl']

In [16]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib

app = Flask(__name__)

# Load trained model & scaler
model = joblib.load("best_xgboost.pkl")
scaler = joblib.load("scaler.pkl")

# Feature Engineering Function
def process_input(data):
    df = pd.DataFrame([data])

    # Compute additional features
    df["Vibration_MA"] = df["Vibration_Level"].rolling(window=3, min_periods=1).mean()
    df["Temp_MA"] = df["Motor_Temperature"].rolling(window=3, min_periods=1).mean()

    # Standardize features
    features = ["Load_Weight", "Motor_Temperature", "Vibration_Level", "Operating_Hours", "Vibration_MA", "Temp_MA"]
    df[features] = scaler.transform(df[features])

    return df[features]

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    processed_data = process_input(data)

    # Make prediction
    prediction = model.predict(processed_data)[0]
    probability = model.predict_proba(processed_data)[0][1]  # Probability of failure

    response = {
        "Failure_Prediction": int(prediction),
        "Failure_Probability": round(probability, 2)
    }
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\tanishka\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime

# Define API endpoint (Flask backend)
API_URL = "http://127.0.0.1:5000/predict"

# Function to simulate real-time crane data
def generate_real_time_data():
    return {
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Load_Weight": round(np.random.uniform(500, 5000), 2),
        "Motor_Temperature": round(np.random.uniform(30, 90), 2),
        "Vibration_Level": round(np.random.uniform(0.2, 5.0), 2),
        "Operating_Hours": round(np.random.uniform(100, 3000), 2),
    }

# Send real-time data to API
while True:
    data = generate_real_time_data()
    response = requests.post(API_URL, json=data)
    print(f"Sent Data: {data}, Response: {response.json()}")
    time.sleep(5)  # Send data every 5 seconds


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001941F4A6A50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))